# 1.0 Setup

In [1]:
!pip install -e "/content/drive/My Drive/Colab Notebooks/Applications of Machine Learning/tic_tac_toe_gym_env/"

Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/Applications%20of%20Machine%20Learning/tic_tac_toe_gym_env
  Running setup.py develop for gym-tictactoe


Once the environment is installed the runtime must be restarted.

# 2.0 Import libraries

In [0]:
import numpy as np
import gym
import gym_tictactoe
import random
from tqdm import tqdm

# 3.0 Create Q-Table

In [0]:
# Makes the environment
env = gym.make('tictactoe-v0')

# Creates an empty dictionary for the q-table
agent = {}

In [0]:
# Creates a dictionary to convert q-table columns to board moves
action_dictionary = {
    0 : (0,0),
    1 : (0,1),
    2 : (0,2),
    3 : (1,0),
    4 : (1,1),
    5 : (1,2),
    6 : (2,0),
    7 : (2,1),
    8 : (2,2)
}

In [0]:
def hash_state(state, q_table):
  """
  Hashes the board state. Generates a unique value for each state for the qtable rows.

  Parameters:
  state (ndarray): An array representing the board state.

  Returns:
  hash_value (int): A value representing the board state.
  """

  #Converts the state to a string
  state_string = state.tostring()

  # Hashes the string
  hash_value = hash(state_string)

  # If the hash value does not exist in the qtable it is added
  if hash_value not in q_table:
    q_table[hash_value] = np.full(9,0, dtype=float)

  return hash_value

In [0]:
def get_q_values(state, q_table):
  """
  Gets the q values for a particular state.

  Parameters:
  state (ndarray): An array representing the board state.

  Returns:
  q_table[hash_state(state)] (ndarray): An array containing q values for a particular state.
  """
  return q_table[hash_state(state, q_table)]

In [0]:
def get_action(code):
  """
  Gets the action corresponding with the code from the dictionary.

  Parameters:
  code (int): Code corresponding with an action in the dictionary.

  Returns:
  action_dictionary[code] (tuple): Coordinates corresponding to board positions.
  """
  return action_dictionary[code]

In [0]:
def get_action_code(action):
  """
  Gets the action code corresponding with the action.

  Parameters:
  code (int): Code corresponding with an action in the dictionary.

  Returns:
  action_dictionary[code] (tuple): Coordinates corresponding to board positions.
  """

  # Iterates through the dictionary elements
  for key, value in action_dictionary.items():
    # Returns the code if the action matches a dictionary element
    if value == action:
      return key

In [0]:
def play_random_turn(state):
  """
  Plays a random turn.

  Parameters:
  state (ndarray): An array representing the board state.

  Returns:
  move
  """

  # Gets the coordinates of valid moves
  valid_moves = np.where(state == 0)

  # Pairs the coordinates values together
  moves = [list(move) for move in zip(valid_moves[0], valid_moves[1])]

  # Creates the list of valid move coordinates
  moves_list = []
  for i in moves:
    move = tuple(i)
    moves_list.append(move)

  # Picks a random move from the list
  move = random.choice(moves_list)
  return move

In [0]:
def bellman_equation(state, new_state, reward, action, q_table):
  q_table[hash_state(state, q_table)][get_action_code(action)] = q_table[hash_state(state, q_table)][get_action_code(action)] + learning_rate * (reward + gamma * np.max(q_table[hash_state(new_state, q_table)]) - q_table[hash_state(state, q_table)][get_action_code(action)])

# 3.0 Train Q-Table

In [0]:
total_episodes = 500000        
total_test_episodes = 1000
max_steps = 9                

learning_rate = 0.5          
gamma = 0.95 # Discount rate        

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.00001          # Exponential decay rate for exploration probability

In [0]:
def train_q_table(q_table):

  # Uses the global variable
  global epsilon
  
  # Resets the environment
  state = env.reset()

  # Creates the first row in the q-table
  q_table[hash_state(state, q_table)] = np.full(9, 0, dtype=float)

  # Iterates through all the episodes
  for episode in tqdm(range(total_episodes), position=0):

    # Resets the environment
    state = env.reset()

    done = False
    
    # Iterates through the steps
    for step in range(max_steps):

      # Checks it is the agent's turn
      if (step % 2) == 0:

        # Generates a random number
        exploitation = random.uniform(0,1)

        # Checks if the exploration rate is bigger than the exploitation value
        if exploitation > epsilon:

          # Performs an action from the q-table
          action = get_action(np.argmax(get_q_values(state, q_table)))

        else:
          # Performs a random action
          action = tuple(env.action_space.sample())

        # Takes a step with the chosen action   
        new_state, reward, done = env.step(action)

        # Uses the Bellman equation to update the q-table
        bellman_equation(state, new_state, reward, action, q_table)

      else:
        # The opponent plays a random turn
        action = play_random_turn(state)

        # Takes a step with the chosen action
        new_state, reward, done = env.step(action)

        if done:
          # Uses the Bellman equation to update the q-table
          bellman_equation(state, new_state, reward, action, q_table)

      # Updates the state
      state = new_state
      
      if done:
        break
    
    # Increments the episode
    episode += 1
    
    # Decays the exploration rate
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

In [12]:
# Trains the agent
train_q_table(agent)

100%|██████████| 500000/500000 [04:10<00:00, 1994.36it/s]


# 4.0 Play Tic-Tac-Toe

In [13]:
%%time
env.reset()
rewards = []
wins = 0

# Iterates through all the episodes
for episode in tqdm(range(total_test_episodes)):

  # Resets the environment
  state = env.reset()
  step = 0
  done = False
  total_reward = 0

  print("**********************************************************")
  print("EPISODE: ", episode)

  # Iterates through the steps
  for step in range(max_steps):
    
    # Checks if it is the agent's turn
    if (step % 2) == 0:

      # Gets the action from the trained q-table
      action = get_action(np.argmax(get_q_values(state, agent)))

      # Takes a step with the chosen action
      new_state, reward, done = env.step(action)

      # Increments the total rewards
      total_reward += reward

    else:

      # The opponent plays a random turn
      action = play_random_turn(state)

      # Takes a step with the chosen action
      new_state, reward, done = env.step(action)
      
      if done:
        # Increments the total rewards
        total_reward += reward
    
    if done:
      rewards.append(total_reward)
      print(state)
      print("Score: ", total_reward)
      break
    
    # Updates the state
    state = new_state

  6%|▌         | 56/1000 [00:00<00:01, 559.01it/s]

**********************************************************
EPISODE:  0
[[ 1.  1.  1.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  1
[[ 1.  1.  1.]
 [ 0.  0. -1.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  2
[[ 1.  1. -1.]
 [ 1.  1.  0.]
 [-1. -1. -1.]]
Score:  -6
**********************************************************
EPISODE:  3
[[ 1.  1.  1.]
 [-1.  0. -1.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  4
[[ 1.  1.  1.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]]
Score:  12
**********************************************************
EPISODE:  5
[[ 1.  1.  1.]
 [ 0.  0. -1.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  6
[[ 1. -1.  1.]
 [ 1. -1.  1.]
 [-1.  1. -1.]]
Score:  -6
**********************************************************
EPISODE:  7
[[ 1.  1.  1.]
 [-1.  0. -1.]
 [ 

 15%|█▌        | 152/1000 [00:00<00:01, 507.87it/s]

Score:  -6
**********************************************************
EPISODE:  107
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1. -1.]]
Score:  12
**********************************************************
EPISODE:  108
[[ 1.  1.  1.]
 [-1.  0. -1.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  109
[[ 1.  1.  1.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  110
[[ 1.  1. -1.]
 [ 1.  1. -1.]
 [-1. -1.  1.]]
Score:  14
**********************************************************
EPISODE:  111
[[ 1.  1.  1.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  112
[[ 1.  1.  1.]
 [ 0. -1. -1.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  113
[[ 1.  1.  1.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  114
[[ 1. 

 24%|██▍       | 243/1000 [00:00<00:01, 476.80it/s]

...

741
[[ 1.  1.  1.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]]
Score:  12
**********************************************************
EPISODE:  742
[[ 1.  1.  1.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]]
Score:  12
**********************************************************
EPISODE:  743
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1. -1.]]
Score:  12
**********************************************************
EPISODE:  744
[[ 1.  1.  1.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  745
[[ 1.  1.  1.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  746
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [-1.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  747
[[ 1.  1.  1.]
 [-1. -1.  0.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  748
[[ 1.  1.  1.]
 [-1. -1.  0.]
 [ 0.  0.  0.]]
Score:  12
*****************************

 96%|█████████▌| 961/1000 [00:01<00:00, 552.85it/s]

EPISODE:  859
[[ 1.  1. -1.]
 [ 1. -1. -1.]
 [ 1.  0.  0.]]
Score:  13
**********************************************************
EPISODE:  860
[[ 1.  1. -1.]
 [ 1.  1. -1.]
 [-1.  0. -1.]]
Score:  -6
**********************************************************
EPISODE:  861
[[ 1.  1. -1.]
 [ 1.  1. -1.]
 [ 0. -1. -1.]]
Score:  -6
**********************************************************
EPISODE:  862
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [-1. -1.  0.]]
Score:  12
**********************************************************
EPISODE:  863
[[ 1.  1. -1.]
 [ 1.  1. -1.]
 [-1. -1.  1.]]
Score:  14
**********************************************************
EPISODE:  864
[[ 1.  1.  1.]
 [-1. -1.  0.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  865
[[ 1.  1.  1.]
 [-1.  0.  0.]
 [ 0. -1.  0.]]
Score:  12
**********************************************************
EPISODE:  866
[[ 1.  1.  1.]
 [ 0. -1. -1.]
 [ 0.  0.  0.]]
Score:  12
*******************

100%|██████████| 1000/1000 [00:01<00:00, 526.70it/s]

Score:  12
**********************************************************
EPISODE:  981
[[ 1.  1.  1.]
 [ 0. -1. -1.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  982
[[ 1.  1.  1.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  983
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [-1.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  984
[[ 1.  1.  1.]
 [-1.  0.  0.]
 [-1.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  985
[[ 1.  1. -1.]
 [ 1.  1.  0.]
 [-1. -1. -1.]]
Score:  -6
**********************************************************
EPISODE:  986
[[ 1.  1.  1.]
 [-1.  0. -1.]
 [ 0.  0.  0.]]
Score:  12
**********************************************************
EPISODE:  987
[[ 1.  1.  1.]
 [ 0.  0. -1.]
 [ 0.  0. -1.]]
Score:  12
**********************************************************
EPISODE:  988
[[ 1. 

In [14]:
losses = 0
for score in rewards:
  if score < 0:
    losses+=1

# Prints the percentage of games lost
print("Loss Percentage: " + str(losses/total_test_episodes *100) + "%")

Loss Percentage: 23.400000000000002%


# 5.0 Agent VS Agent

In [0]:
# Creates the agents
agent1 = {}
agent2 = {}

# Sets the matches and score counters
matches = 500
agent1wins = 0
agent2wins = 0
draws = 0

In [16]:
# Trains the agents
train_q_table(agent1)
train_q_table(agent2)

100%|██████████| 500000/500000 [04:13<00:00, 1974.03it/s]


In [17]:
# Resets the environment
env.reset()

# Iterates through all the matches
for matches in range(matches):

  state = env.reset()
  step = 0
  done = False

  print("**********************************************************")
  print("MATCH: ", matches)

  #Iterates through the steps
  for step in range(max_steps):
    
    # Checks if it is the first agent's turn
    if (step % 2) == 0:

      # Gets the action from the agent1's trained q-table
      action = get_action(np.argmax(get_q_values(state, agent1)))

      # Takes a step with the chosen action
      new_state, reward, done = env.step(action)

      # Records wins and draws
      if done:
        if reward == 10:
          agent1wins +=1
        else:
          draws+=1

    else:

      # Gets the action from the agent1's trained q-table
      action = get_action(np.argmax(get_q_values(state, agent2)))

      # Takes a step with the chosen action
      new_state, reward, done = env.step(action)

      # Records wins and draws
      if done:
        if reward == 10:
          agent2wins +=1
        else:
          draws+=1
    
    if done:
      print(state)
      break
    
    # Updates the state
    state = new_state

**********************************************************
MATCH:  0
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  1
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  2
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  3
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  4
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  5
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  6
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  7
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************************
MATCH:  8
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
**********************************************

In [18]:
print("Agent 1 Wins: {} Agent 2 Wins: {} Draws: {}".format(agent1wins, agent2wins, draws))

Agent 1 Wins: 0 Agent 2 Wins: 0 Draws: 500


# 6.0 Human VS Agent

In [0]:
state = env.reset()

Run this cell to have the agent take a turn:

In [0]:
action = get_action(np.argmax(get_q_values(state, agent1)))
new_state, reward, done = env.step(action)
print(state)

[[ 1.  1. -1.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]


Enter the coordinates into env.step() and run this cell to take your turn:

In [0]:
new_state, reward, done = env.step((1,2))
print(state)

[[ 1.  1. -1.]
 [ 1.  0. -1.]
 [ 0.  0. -1.]]
